<a href="https://colab.research.google.com/github/KTH-EXPECA/examples/blob/main/ep5g_edge_public_telenor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [ ]:
import os, re
from getpass import getpass

with open('demo_project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

Install required packages and dependencies. Ignore the warnings.

In [ ]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

Import packages

In [ ]:
import json
from loguru import logger
import chi.network, chi.container
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed, show_reservation_byname, get_available_publicips, get_segment_ids, get_worker_interfaces

In the next cell, we reserve the required equipment and resources to form an end to end experiment setup. We reserve EP5G network.

In [ ]:
# ep5g reservation
ep5g_lease = show_reservation_byname("ep5g-lease")
segment_ids = get_segment_ids('ep5g')
if not ep5g_lease:
    ep5g_lease = reserve(
        { "type":"network", "name": "ep5g", "net_name": "ep5g-vip", "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )


leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

In the following section we setup the networking equipment for ep5g, edge-net, Telenor Edge, and Internet.

In [ ]:
# create edge-net
edgenet = chi.network.create_network("edge-net")
chi.network.create_subnet("edge-net-subnet",edgenet["id"],"10.70.70.0/24",gateway_ip=None,enable_dhcp=False)
logger.success("edge-net is created.")

# create telenor-int-net
telintnet = chi.network.create_network("telenor-int-net")
chi.network.create_subnet("telenor-int-subnet",telintnet["id"],"10.80.80.0/24",gateway_ip=None,enable_dhcp=False)
logger.success("telenor-int-net is created.")

# create ep5g-vip-router
router = chi.network.create_router("ep5g-vip-router", "public")
logger.success("ep5g-vip-router router is created.")
logger.info(f"{json.dumps(router,indent=4)}")

# connect ep5g-vip-net to ep5g-vip-router
ep5gnet = chi.network.get_network("ep5g-vip-net")
portadd = chi.network.add_subnet_to_router(router["id"],ep5gnet["subnets"][0])
logger.success("An interface on ep5g-vip-net is added to the router")

# connect edge-net to ep5g-vip-router
edgenet = chi.network.get_network("edge-net")
portadd = chi.network.add_subnet_to_router(router["id"],edgenet["subnets"][0])
logger.success("An interface on edge-net is added to the router")

# connect telenor-int-net to ep5g-vip-router
telintnet = chi.network.get_network("telenor-int-net")
port = chi.network.create_port(
    "telenor-int-port1",  # Name of the port
    network_id=telintnet["id"],
    fixed_ips=[{"subnet_id": telintnet["subnets"][0], "ip_address": "10.80.80.1"}]  # Specify new IP address
)
port_id = port["port"]["id"]
chi.network.add_port_to_router(router["id"], port_id)
logger.success("An interface on telenor-int-net is added to the router")

# add ep5g route to ep5g-vip-router
routeadd = chi.network.add_route_to_router(router["id"],"172.16.0.0/16","10.30.111.10")
logger.success("the ep5g route added to the router")

# add telenor route to ep5g-vip-router, which goes to Telenor router
routeadd = chi.network.add_route_to_router(router["id"],"10.2.58.0/24","10.80.80.2")
logger.success("the telenor route added to the router")

# create telenor-int-router
router = chi.network.create_router("telenor-int-router", "telenor-net")
logger.success("telenor-int-router router is created.")
logger.info(f"{json.dumps(router,indent=4)}")

# connect telenor-int-net to telenor-int-router
telintnet = chi.network.get_network("telenor-int-net")
port = chi.network.create_port(
    "telenor-int-port2",  # Name of the port
    network_id=telintnet["id"],
    fixed_ips=[{"subnet_id": telintnet["subnets"][0], "ip_address": "10.80.80.2"}]  # Specify new IP address
)
port_id = port["port"]["id"]
chi.network.add_port_to_router(router["id"], port_id)
logger.success("An interface on telenor-int-net is added to the router")

# add ep5g route to telenor-router
routeadd = chi.network.add_route_to_router(router["id"],"172.16.0.0/16","10.80.80.1")
logger.success("the ep5g route added to the router")

Cleanup of containers, routers, networks, and leases.

In [ ]:
container_names = []
router_names = []
network_names = []
lease_names = []

In [ ]:
for container_name in container_names:
    try:
        chi.container.delete_container(container_name, force=True)
        print(f"Deleted container: {container_name} forcefully")
    except Exception as e:
        print(f"Failed to delete container {container_name}: {e}")

In [ ]:
for router_name in router_names:
    try:
        # Get the router details
        router = chi.network.get_router(router_name)
        if not router:
            print(f"Router not found: {router_name}")
            continue

        # Delete all static routes associated with the router
        routes = chi.network.list_router_routes(router_name)
        for route in routes:
            chi.network.delete_router_route(router_name, route["id"])
            print(f"Deleted static route: {route['destination']} for router: {router_name}")

        # Remove all interfaces from the router
        interfaces = chi.network.list_router_interfaces(router_name)
        for interface in interfaces:
            chi.network.remove_router_interface(router_name, interface["port_id"])
            print(f"Removed interface: {interface['port_id']} from router: {router_name}")

        # Finally, delete the router
        chi.network.delete_router(router_name)
        print(f"Deleted router: {router_name}")

    except Exception as e:
        print(f"Failed to delete router {router_name}: {e}")

In [ ]:
for network_name in network_names:
    try:
        network = chi.network.get_network(network_name)
        if network:
            chi.network.delete_network(network_name)
            print(f"Deleted network: {network_name}")
        else:
            print(f"Network not found: {network_name}")
    except Exception as e:
        print(f"Failed to delete network {network_name}: {e}")

In [ ]:
for lease_name in lease_names:
    try:
        # Retrieve the lease by name
        lease = chi.lease.get_lease_by_name(lease_name)
        if lease:
            chi.lease.delete_lease(lease["id"])
            print(f"Deleted lease: {lease_name}")
        else:
            print(f"Lease not found: {lease_name}")
    except Exception as e:
        print(f"Failed to delete lease {lease_name}: {e}")